# Day 1 - Lab 1: AI-Powered Requirements & User Stories (Solution)

**Objective:** Use a Large Language Model (LLM) to decompose a vague problem statement into structured features, user personas, and Agile user stories, culminating in a machine-readable JSON artifact.

**Introduction:**
This notebook contains the complete solution for Lab 1. It demonstrates how to use an LLM to systematically break down a problem, generate structured requirements, and programmatically validate the output. Each step includes explanations of the code and the reasoning behind the prompts.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

**Purpose:** This initial block of code prepares our environment for the lab. It adds the project root to the system path to ensure our `utils.py` helper script can be imported, and then initializes the LLM API client.

**Model Selection:**
Our `utils.py` script is configured to work with multiple AI providers. You can change the `model_name` parameter in the `setup_llm_client()` function to any of the models listed in the `RECOMMENDED_MODELS` dictionary in `utils.py`. For example, to use a Hugging Face model, you could change the line to: `client, model_name, api_provider = setup_llm_client(model_name="meta-llama/Llama-3.3-70B-Instruct")`

**Libraries Explained:**
- **`os`**, **`sys`**: Standard Python libraries for interacting with the file system and Python's path, ensuring our modules are discoverable.
- **`json`**: A standard library for working with JSON data. We use `json.loads` to parse the LLM's text output into a Python dictionary or list, and `json.dumps` to format Python objects into a pretty-printed JSON string for saving.
- **`utils`**: Our custom helper script. 
  - `setup_llm_client()`: Handles reading the `.env` file and initializing the API client.
  - `get_completion()`: Simplifies the process of sending a prompt to the LLM and receiving a text response.
  - `save_artifact()`: Ensures our project artifacts are stored consistently in the `artifacts` directory.
  - `clean_llm_output()`: A new standardized function to remove markdown fences from LLM outputs.

In [6]:
import sys
import os
import json

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    # Assumes the notebook is in 'labs/Day_01_.../'
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    # Fallback for different execution environments
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, clean_llm_output

# Initialize the LLM client. You can change the model here.
# For example: setup_llm_client(model_name="gemini-2.5-flash")
client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

✅ LLM Client configured: Using 'openai' with model 'gpt-4o'


## Step 2: The Problem Statement

We define our starting point—a simple, high-level problem statement—as a Python variable. This makes it easy to reuse in multiple prompts.

In [7]:
problem_statement = "We need a tool to help our company's new hires get up to speed."

## Step 3: The Challenges

Here are the complete solutions for each challenge.

### Challenge 1 (Foundational): Brainstorming Features

**Explanation:**
This first challenge is about exploration. We use simple, direct prompts to get the LLM's initial thoughts on the problem. The goal is to generate a broad set of ideas (features and personas) that will serve as the raw material for the more structured tasks to follow. We expect the output to be human-readable markdown.

In [8]:
# This prompt is direct and open-ended, encouraging the LLM to be creative.
features_prompt = f"""
Based on the problem statement: '{problem_statement}', brainstorm a list of potential features for a new hire onboarding tool. 
Format the output as a simple markdown list.
"""

print("--- Brainstorming Features ---")
brainstormed_features = get_completion(features_prompt, client, model_name, api_provider)
print(brainstormed_features)

# This prompt asks for specific roles to ground the brainstorming in user-centric thinking.
personas_prompt = f"""
Based on the problem statement: '{problem_statement}', identify and describe three distinct user personas who would interact with this tool. 
For each persona, describe their role and main goal.
"""

print("\n--- Identifying User Personas ---")
user_personas = get_completion(personas_prompt, client, model_name, api_provider)
print(user_personas)

--- Brainstorming Features ---
- **Welcome Dashboard**
  - Personalized welcome message
  - Overview of the onboarding process
  - Quick links to essential resources

- **Task Checklist**
  - Step-by-step onboarding tasks
  - Progress tracking
  - Due dates and reminders

- **Document Repository**
  - Access to company policies and manuals
  - Downloadable forms and templates
  - Version control for document updates

- **Interactive Training Modules**
  - Video tutorials and webinars
  - Quizzes and assessments
  - Gamified learning elements

- **Mentorship Program Integration**
  - Automated mentor matching
  - Scheduled mentor meetings
  - Feedback and evaluation forms

- **Company Culture Introduction**
  - Videos introducing company values and mission
  - Employee testimonials and success stories
  - Virtual office tours

- **Communication Tools**
  - Integrated chat with HR and team members
  - FAQ section and helpdesk access
  - Scheduled Q&A sessions with leadership

- **Feedbac

### Challenge 2 (Intermediate): Generating Formal User Stories

**Explanation:**
This challenge represents a significant increase in complexity and value. We are no longer asking for simple text; we are demanding a specific, structured data format (JSON). 

The prompt is carefully engineered:
1.  **Persona:** `You are a Senior Product Manager...` tells the LLM the role it should adopt.
2.  **Context:** We provide the previous outputs (`problem_statement`, `brainstormed_features`, `user_personas`) inside `<context>` tags to give the LLM all the necessary information.
3.  **Format:** The `OUTPUT REQUIREMENTS` section is extremely explicit. It tells the LLM to *only* output JSON, defines the exact keys for each object, and specifies the format for nested data (like the array of Gherkin strings). This strictness is key to getting reliable, machine-readable output.
4.  **Parsing:** The `try...except` block is a crucial step. It attempts to parse the LLM's string output into a Python list of dictionaries. If it succeeds, we know the LLM followed our instructions perfectly. If it fails, we print the raw output to help debug the prompt.

In [11]:
# The prompt is highly-structured to guide the LLM toward a perfect JSON output.
json_user_stories_prompt = f"""
You are a Senior Product Manager creating a product backlog for a new hire onboarding tool.

Based on the following context:
<context>
Problem Statement: {problem_statement}
Potential Features: {brainstormed_features}
User Personas: {user_personas}
</context>

Your task is to generate a list of 5 detailed user stories.

**OUTPUT REQUIREMENTS**:
- You MUST output a valid JSON array. Your response must begin with [ and end with ]. Do not include any text or markdown before or after the JSON array.
- Each object in the array must represent a single user story.
- Each object must have the following keys: 'id' (an integer), 'persona' (a string from the personas), 'user_story' (a string in the format 'As a [persona], I want [goal], so that [benefit].'), and 'acceptance_criteria' (an array of strings, with each string in Gherkin format 'Given/When/Then').
"""

print("--- Generating User Stories as JSON ---")
# We set a lower temperature to encourage the LLM to stick to the requested format.
json_output_str = get_completion(json_user_stories_prompt, client, model_name, api_provider, temperature=0.2)

# Attempt to parse the string output into a Python list.
try:
    # Use our new standardized cleaning function from utils.py
    cleaned_json_str = clean_llm_output(json_output_str, language='json')
    
    user_stories_json = json.loads(cleaned_json_str)
    print("Successfully parsed LLM output as JSON.")
    # Pretty-print the first user story to verify its structure
    print("\n--- Sample User Story ---")
    print(json.dumps(user_stories_json[0], indent=2))
except (json.JSONDecodeError, TypeError, IndexError) as e:
    print(f"Error: Failed to parse LLM output as JSON. Error: {e}")
    print("LLM Output was:\n", json_output_str)
    user_stories_json = [] # Assign an empty list to prevent errors in the next cell

--- Generating User Stories as JSON ---
Successfully parsed LLM output as JSON.

--- Sample User Story ---
{
  "id": 1,
  "persona": "New Hire (Entry-Level Employee)",
  "user_story": "As a New Hire (Entry-Level Employee), I want a personalized welcome dashboard, so that I can quickly access essential resources and understand the onboarding process.",
  "acceptance_criteria": [
    "Given I am a new hire, When I log into the onboarding tool, Then I should see a personalized welcome message.",
    "Given I am on the welcome dashboard, When I look for resources, Then I should see quick links to essential resources.",
    "Given I am on the welcome dashboard, When I want to understand the onboarding process, Then I should see an overview of the onboarding process."
  ]
}


### Challenge 3 (Advanced): Programmatic Validation and Artifact Creation

**Explanation:**
This is the final and most critical step. We treat the LLM's output as untrusted input and subject it to programmatic validation. This ensures that the artifact we create is reliable and can be consumed by other automated tools in later stages of the SDLC without causing errors. 

The `validate_and_save_stories` function acts as a gatekeeper. It checks for the correct data types (a list of objects) and ensures that all required fields are present in each object. Only if all checks pass do we proceed to save the file using `save_artifact`. This creates a trustworthy `day1_user_stories.json` file that can be confidently used as an input for other automated processes in our SDLC.

In [5]:
def validate_and_save_stories(stories_data):
    """Validates the structure of the user stories data and saves it if valid."""
    if not isinstance(stories_data, list) or not stories_data:
        print("Validation Failed: Data is not a non-empty list.")
        return

    required_keys = ['id', 'persona', 'user_story', 'acceptance_criteria']
    all_stories_valid = True

    # Loop through each story object in the list.
    for i, story in enumerate(stories_data):
        # Check for the presence of all required keys.
        if not all(key in story for key in required_keys):
            print(f"Validation Failed: Story at index {i} is missing one or more required keys.")
            all_stories_valid = False
            continue # Don't bother with further checks for this invalid story
        
        # Check that the acceptance criteria is a list with at least one item.
        ac = story.get('acceptance_criteria')
        if not isinstance(ac, list) or not ac:
            print(f"Validation Failed: Story at index {i} ('{story.get('id')}') has invalid or empty acceptance criteria.")
            all_stories_valid = False

    # Only save the artifact if all stories in the list are valid.
    if all_stories_valid:
        print("\nAll user stories passed validation.")
        artifact_path = "artifacts/day1_user_stories.json"
        
        # Use the helper function to save the file, creating the 'artifacts' directory if needed.
        # We use json.dumps with an indent to make the saved file human-readable.
        save_artifact(json.dumps(stories_data, indent=2), artifact_path)
    else:
        print("\nValidation failed. Artifact not saved.")

# Run the validation function on the data we parsed from the LLM.
if 'user_stories_json' in locals() and user_stories_json:
    validate_and_save_stories(user_stories_json)
else:
    print("Skipping validation as user_stories_json is empty or could not be parsed.")


All user stories passed validation.
✅ Successfully saved artifact to: artifacts/day1_user_stories.json


## Lab Conclusion

Congratulations! You have completed the first lab. You started with a vague, one-sentence problem and finished with a structured, validated, machine-readable requirements artifact. This is the critical first step in an AI-assisted software development lifecycle. The `day1_user_stories.json` file you created will be the direct input for our next lab, where we will generate a formal Product Requirements Document (PRD).

> **Key Takeaway:** The single most important skill demonstrated in this lab is turning unstructured ideas into structured, machine-readable data (JSON). This transformation is what enables automation and integration with other tools later in the SDLC.